In [46]:
from IPython.display import display
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from enum import Enum
from copy import deepcopy
from pprint import pprint
from functools import reduce
from time import time
from queue import PriorityQueue

def print_mat(mat):
    for row in mat:
        print(''.join(row))

files = [f for f in listdir('.') if isfile(join('.', f))]
pat = re.compile('(ex(\d)+.txt)|(in(\d)*.txt)')
files = [f for f in files if pat.match(f)]

box = widgets.Select(
    options=files,
    value='ex1.txt',
    rows=4,
    description='Input file:',
    disabled=False
)

display(box)


Select(description='Input file:', index=2, options=('in.txt', 'ex2.txt', 'ex1.txt'), rows=4, value='ex1.txt')

## Parse file

In [96]:
name = box.value
f = open(name, 'r')
lines = f.readlines()
lines = list(map(lambda l: l.rstrip(), lines))


# TODO: Parse the input

binary = "".join(map(lambda h: format(int(h, base=16), "04b"), list(lines[0])))

class Packet:
    def __init__(self, version, ptype, subpackets=[], value=None):
        self.version = version
        self.ptype = ptype
        self.subpackets = subpackets
        self.value = value

print("Done parsing \"{name}\"".format(name=name))


Done parsing "in.txt"


### Part 1

In [97]:
def ppacket(input):
    v = int(input[:3], base=2)
    t = int(input[3:6], base=2)

    if t == 4:
        i = 6
        value = ''
        while input[i] != '0':
            value += input[i+1:i+5]
            i += 5
        value += input[i+1:i+5]
        return (Packet(v, t, value=int(value, base=2)), input[i+5:])
    else:
        if input[6] == '0':
            l = int(input[7:22], base=2)
            subpackets = []
            rest = input[22:22+l]
            while len(rest) > 0:
                p, rest = ppacket(rest)
                subpackets.append(p)
            return (Packet(v, t, subpackets=subpackets), input[22+l:])
        else:
            n = int(input[7:18], base=2)
            subpackets = []
            rest = input[18:]
            for _ in range(n):
                p, rest = ppacket(rest)
                subpackets.append(p)
            return (Packet(v, t, subpackets=subpackets), rest)


def count(packet):
    return packet.version + sum(map(count, packet.subpackets))


p, _ = ppacket(binary)
print(count(p))


963


### Part 2

In [98]:
from math import prod


def value(packet):
    match packet.ptype:
        case 0:
            return sum(map(value, packet.subpackets))
        case 1:
            return prod(map(value, packet.subpackets))
        case 2:
            return min(map(value, packet.subpackets))
        case 3:
            return max(map(value, packet.subpackets))
        case 4:
            return packet.value
        case 5:
            V = list(map(value, packet.subpackets))
            return V[0] > V[1]
        case 6:
            V = list(map(value, packet.subpackets))
            return V[0] < V[1]
        case 7:
            V = list(map(value, packet.subpackets))
            return V[0] == V[1]


print(int(value(p)))


1549026292886
